# Imports

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import optuna

from sklearn.ensemble import ExtraTreesClassifier

import warnings
import gc
warnings.simplefilter('ignore')

KAGGLE_DIR = r'../input/tabular-playground-series-feb-2022/'
LOCAL_DIR = r''
KAGGLE = True
RS = 69420

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

# Preprocessing

In [ ]:
%%time
if KAGGLE:
    print(f"{'*'*10} Loading Training Data... {'*'*10}")
    df = pd.read_csv(KAGGLE_DIR+'train.csv', index_col=0).pipe(reduce_mem_usage)
    print(f"{'*'*10} Loading Testing Data... {'*'*10}")
    test = pd.read_csv(KAGGLE_DIR+'test.csv', index_col=0).pipe(reduce_mem_usage)
    sub = pd.read_csv(KAGGLE_DIR+'sample_submission.csv').pipe(reduce_mem_usage)
else:
    print(f"{'*'*10} Loading Training Data... {'*'*10}")
    df = pd.read_csv(LOCAL_DIR+'train.csv', index_col=0).pipe(reduce_mem_usage)
    print(f"{'*'*10} Loading Testing Data... {'*'*10}")
    test = pd.read_csv(LOCAL_DIR+'test.csv', index_col=0).pipe(reduce_mem_usage)
    sub = pd.read_csv(LOCAL_DIR+'sample_submission.csv').pipe(reduce_mem_usage)

In [ ]:
lb = LabelEncoder()
X = df.iloc[:, :-1].values
y = lb.fit_transform(df['target'])

In [ ]:
X.shape, y.shape

In [ ]:
del df
gc.collect()

# Simple Model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=RS)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
clf = ExtraTreesClassifier(
    n_estimators=100,
    n_jobs=-1,
    random_state=RS,
    verbose=1
)

In [ ]:
%%time
clf.fit(
    X_train, y_train,
)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
acc = accuracy_score(y_pred, y_test)
print(f"Model Accuracy: {round(acc, 6)}")

# Optuna Hyperparameter Tuning

In [ ]:
def objective(trial, X, y):

    param = {
        #'verbose': trial.suggest_categorical('verbosity', [1]),
        'random_state': trial.suggest_categorical('random_state', [RS]),
        'n_jobs': trial.suggest_categorical('n_jobs', [-1]),
        'n_estimators':trial.suggest_int('n_estimators', 100, 5000, step=100),
        'max_features':trial.suggest_int('max_features', 1, 286, step=2),
        'min_samples_split':trial.suggest_int('min_samples_split', 2, 20)
    }

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=RS)

    clf = ExtraTreesClassifier(**param)
    
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    
    acc = accuracy_score(y_pred, y_test)
    loss = round(acc, 6)
    return loss

In [ ]:
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(),
    pruner=optuna.pruners.HyperbandPruner(),
    study_name='ExtraTrees-Hyperparameter-Tuning'
)

In [ ]:
func = lambda trial: objective(trial, X, y)

In [ ]:
default_params = {
        'verbose':1,
        'random_state':RS,
        'n_jobs':-1,
        'n_estimators':100,
        'max_features':round(np.sqrt(X.shape[1])),
        'min_samples_split':2
}

study.enqueue_trial(default_params)

In [ ]:
%%time
study.optimize(
    func,
#     n_trials=10,
    timeout=3600*3,
    gc_after_trial=True
)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
print(f'Best value (Accuracy): {study.best_value:.5f}')
print(f'Best params:')

for key, value in study.best_params.items():
    print(f'\t{key}: {value}')
    
    best_params = study.best_params
    
best_params